In [2]:
%pip install google-generativeai python-dotenv
%pip install --upgrade jupyter ipywidgets
%pip install --upgrade transformers datasets[audio] accelerate
%pip install --upgrade pip 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
import google.generativeai as genai

from dotenv import load_dotenv

import os
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

In [1]:
import google.generativeai as genai

from dotenv import load_dotenv

import os
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

audio_file_path = r"C:\Users\전산2실 PC\Downloads\PRACTICE1\chap5\lsy_audio_2023_58s.mp3"
audio_file= genai.upload_file(audio_file_path)
model = genai.GenerativeModel("gemini-2.5-pro")
response = model.generate_content(["이 오디오 파일을 텍스트로 변환해줘", audio_file])

#print(response)
print(response.text)

안녕하세요. 어 이 강의는 GPT API로 챗봇 만들기라는 내용을 다루는 강의입니다. 어 GPT API에 대해서 뭐 생소하신 분들도 있을 텐데 어 우리가 잘 알고 있는 챗 GPT, 챗 GPT를 이용해, 챗 GPT의 기능을 이용해서 어 우리가 원하는 프로그램을 어떻게 만드는지에 대해서 어 이야기할 거예요. 그래서 뭐 이런 강의들이 사실 많이 있습니다. 그래서 뭐 여러 가지들이 있는데 좀 이 강의의 특징이라고 한다면 이제 GPT로 명확한 미션을 달성하는 챗봇 프로그램을 만드는 게 사실 쉽지는 않은데 이거를 어떻게 해서 구현을 하는지 그리고 그게 왜 필요한지에 대해서 좀 이야기를 할 거고요. 어 그 예제로 뭐 예제는 여러 가지가 될 수 있는데 여기서 예제로 하는 거는 음악 플레이리스트 동영상을 자동으로 대화를 통해서 생성하는 프로그램을 만드는 거를 다루려고 합니다. 네, 그래서 프로그램이 실행되는 모습을 한번 보여드릴게요. 그 우리가 그 만들 프로그램은 이런 식으로 이제 나타나게 되


In [2]:
import google.generativeai as genai

from dotenv import load_dotenv

import os
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

audio_file_path = r"C:\Users\전산2실 PC\Downloads\PRACTICE1\chap5\lsy_audio_2023_58s.mp3"
audio_file= genai.upload_file(audio_file_path)
model = genai.GenerativeModel("gemini-2.5-pro")
response = model.generate_content(["이 오디오 파일을 영어로 번역하여 텍스트로 작성해줘", audio_file])

#print(response)
print(response.text)

Hello. Uh, this lecture is about creating a chatbot with the GPT API. Uh, some of you may be unfamiliar with the GPT API, but we're going to talk about how to create the programs we want using the features of the well-known ChatGPT.

So, there are actually many lectures like this. So, while there are various ones, a special feature of this lecture is that creating a chatbot program with GPT that accomplishes a clear mission is actually not easy, so we will discuss how to implement this and why it's necessary.

Uh, as an example, and there could be many, the example we'll cover here is creating a program that automatically generates music playlist videos through conversation.

Yes, so let me show you how the program runs. The program we are going to create will appear like this.


In [7]:

import os
os.environ["PATH"] += os.pathsep + r"C:\Users\전산2실 PC\Downloads\PRACTICE1\ffmpeg\bin"

In [1]:
# Gemini + Whisper + CSV 저장 스크립트 (리팩토링)
# 1. 필요한 라이브러리 임포트
import os
import torch
import pandas as pd
import librosa  # 오디오 파일을 직접 로드하기 위해 사용
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

def transcribe_and_save(audio_path, output_csv_path):
    """
    오디오 파일을 텍스트로 변환하고, 타임스탬프와 함께 CSV 파일로 저장합니다.

    Args:
        audio_path (str): 처리할 오디오 파일의 경로
        output_csv_path (str): 결과를 저장할 CSV 파일의 경로
    """
    # 2. 오디오 파일 존재 여부 확인
    if not os.path.exists(audio_path):
        print(f"오류: 오디오 파일을 찾을 수 없습니다. 경로: {audio_path}")
        return

    # 3. 실행 환경 설정 (GPU 또는 CPU)
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    print(f"사용 장치: {device}")

    # 4. Whisper 모델 및 프로세서 로드
    model_id = "openai/whisper-large-v3-turbo"
    print(f"'{model_id}' 모델을 로드합니다...")
    try:
        model = AutoModelForSpeechSeq2Seq.from_pretrained(
            model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
        )
        model.to(device)
        processor = AutoProcessor.from_pretrained(model_id)
    except Exception as e:
        print(f"모델 로드 중 오류 발생: {e}")
        return

    # 5. 음성 인식 파이프라인 생성
    print("음성 인식 파이프라인을 생성합니다...")
    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
        return_timestamps=True,  # 청크별 타임스탬프 반환
        chunk_length_s=10,       # 입력 오디오를 10초씩 나누기
        stride_length_s=2,       # 2초씩 겹치도록 청크 나누기
    )

    # 6. Librosa를 사용해 오디오 파일 직접 로드 (가장 안정적인 방식)
    # 📌 이 부분이 torchcodec 오류를 피하는 핵심입니다.
    print(f"오디오 파일을 로드합니다: {audio_path}")
    try:
        # Whisper 모델의 필요 샘플링 레이트인 16000Hz로 오디오를 로드
        audio_input, sampling_rate = librosa.load(audio_path, sr=16000)
    except Exception as e:
        print(f"오디오 파일 로드 중 오류 발생: {e}")
        return

    # 7. 음성 인식 실행
    print("음성 인식을 시작합니다... (시간이 다소 걸릴 수 있습니다)")
    try:
        # 📌 파일 경로(str)가 아닌, librosa로 읽은 데이터(array)를 전달합니다.
        result = pipe(audio_input)
        print("음성 인식이 완료되었습니다.")
    except Exception as e:
        print(f"음성 인식 중 오류 발생: {e}")
        return

    # 8. 결과를 데이터프레임으로 변환하고 CSV로 저장
    try:
        chunks = result.get("chunks", [])
        if not chunks:
            print("결과에서 타임스탬프 정보를 찾을 수 없습니다.")
            print("전체 텍스트:", result.get("text", "N/A"))
            return
            
        start_end_text = [
            [chunk["timestamp"][0], chunk["timestamp"][1], chunk["text"]]
            for chunk in chunks
        ]

        df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
        
        # CSV 파일 저장
        df.to_csv(output_csv_path, index=False, sep="|")
        print(f"\n결과가 '{output_csv_path}' 파일에 성공적으로 저장되었습니다.")

        # 9. (display 대신) print를 사용해 결과 미리보기 출력
        print("\n--- 결과 미리보기 ---")
        print(df)
        print("--------------------\n")

    except Exception as e:
        print(f"결과 처리 또는 파일 저장 중 오류 발생: {e}")

# --- 스크립트 실행 지점 ---
if __name__ == "__main__":
    # 📌 처리할 오디오 파일과 저장할 파일 경로를 여기서 수정하세요.
    # Windows 경로의 '\' 문자가 문제를 일으키지 않도록 문자열 앞에 'r'을 붙여줍니다.
    input_file = r"C:\Users\전산2실 PC\Downloads\PRACTICE1\chap5\lsy_audio_2023_58s.mp3"
    output_file = "lsy_audio_2023_58.csv"
    
    transcribe_and_save(input_file, output_file)

c:\Users\전산2실 PC\Downloads\PRACTICE1\venv_new\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


사용 장치: cpu
'openai/whisper-large-v3-turbo' 모델을 로드합니다...


`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


음성 인식 파이프라인을 생성합니다...
오디오 파일을 로드합니다: C:\Users\전산2실 PC\Downloads\PRACTICE1\chap5\lsy_audio_2023_58s.mp3


Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


음성 인식을 시작합니다... (시간이 다소 걸릴 수 있습니다)
음성 인식이 완료되었습니다.

결과가 'lsy_audio_2023_58.csv' 파일에 성공적으로 저장되었습니다.

--- 결과 미리보기 ---
    start    end                                               text
0    0.00   6.30     안녕하세요. 이 강의는 GPT-API로 챗봇 만들기 라는 내용을 다루는 강의입니다.
1    7.18  10.00                         GPT-API에 대해서 생소하신 분들도 있을텐데
2   11.00  20.00   우리가 잘 알고 있는 ChatGPT ChatGPT 기능을 이용해서 우리가 원하는 ...
3   20.00  22.00                                          이야기할 거예요.
4   22.00  24.00                            그래서 이런 강의들이 사실 많이 있습니다.
5   24.00  27.48                      그래서 여러가지들이 있는데 이 강의 특징이라고 한다면
6   27.48  29.58                                  GPT로 명확한 미션을 달성하는
7   29.58  31.66                                   챕터 프로그램을 만드는게 사실
8   31.66  33.00                                            쉽지는 않은데
9   33.00  35.16                                  이걸 어떻게 해서 구현을 하는지
10  35.16  36.40                                 그리고 그게 왜 필요한지에 대해서
11  36.40  37.36                                      좀 이야기를 할 거고요.
